In [2]:
import numpy as np
import pandas as pd
import wfdb
import matplotlib.pyplot as plt
import ast
import os
import tensorflow as tf
#from keras.utils import to_categorical

#import tensorflow_datasets as tfds
import datetime

from sklearn.utils import resample
from sklearn.model_selection import train_test_split
from tensorflow_addons.optimizers import CyclicalLearningRate
#from keras.layers import LSTM

In [3]:
def _parse_function(tf_data):
  features = {"data": tf.io.FixedLenFeature((), tf.string),
              "label": tf.io.FixedLenFeature((), tf.string)}
  parsed_features = tf.io.parse_single_example(tf_data, features)
  data = tf.io.parse_tensor(parsed_features['data'], out_type = tf.float32)
  labels = tf.io.parse_tensor(parsed_features['label'], out_type = tf.float32)
  return data, labels

def read_from_path(tf_record_path):
    dataset = tf.data.TFRecordDataset(tf_record_path)
    dataset = dataset.map(_parse_function)
    return dataset

In [4]:
tf_record_train_path = '/Users/xianghuihuang/Desktop/tfrecord_2labels/train.tfrecords'
tf_record_test_path = '/Users/xianghuihuang/Desktop/tfrecord_2labels/test.tfrecords'
tf_record_val_path = '/Users/xianghuihuang/Desktop/tfrecord_2labels/val.tfrecords'
#TODO LIST:
# read train dataset here
train_dataset = read_from_path(tf_record_train_path)
# read test dataset here
test_dataset = read_from_path(tf_record_test_path)
# read validation dataset here
val_dataset = read_from_path(tf_record_val_path)

In [5]:
initial_learning_rate = 0.1
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate, decay_steps=100000, decay_rate=0.96, staircase=True) 
optimizer_RMS_schedule = tf.keras.optimizers.RMSprop(learning_rate=lr_schedule)
optimizer_adam = tf.keras.optimizers.Adam(learning_rate=0.00001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=False, name='Adam')
optimizer_adam_schedule = tf.keras.optimizers.Adam(learning_rate = lr_schedule)
callback = tf.keras.callbacks.EarlyStopping(monitor="val_loss",patience = 15)

In [11]:
#ANN modeling
ann_model = tf.keras.models.Sequential()
ann_model.add(tf.keras.Input(shape=(5000,12)))
ann_model.add(tf.keras.layers.BatchNormalization())
ann_model.add(tf.keras.layers.Dense(50,activation = 'relu'))
ann_model.add(tf.keras.layers.Dense(50,activation = 'relu'))
#ann_model.add(tf.keras.layers.BatchNormalization())
ann_model.add(tf.keras.layers.Dense(50,activation = 'relu'))
ann_model.add(tf.keras.layers.Dropout(0.3))
#ann_model.add(tf.keras.layers.BatchNormalization())
ann_model.add(tf.keras.layers.Dense(25,activation = 'relu'))
ann_model.add(tf.keras.layers.Dropout(0.3))
#ann_model.add(tf.keras.layers.BatchNormalization())
ann_model.add(tf.keras.layers.GlobalAveragePooling1D())
ann_model.add(tf.keras.layers.Dense(12,activation = 'relu'))
ann_model.add(tf.keras.layers.Dropout(0.3))
ann_model.add(tf.keras.layers.Dense(5,activation = 'softmax'))

In [12]:
ann_model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_1 (Batch (None, 5000, 12)          48        
_________________________________________________________________
dense_6 (Dense)              (None, 5000, 50)          650       
_________________________________________________________________
dense_7 (Dense)              (None, 5000, 50)          2550      
_________________________________________________________________
dense_8 (Dense)              (None, 5000, 50)          2550      
_________________________________________________________________
dropout_3 (Dropout)          (None, 5000, 50)          0         
_________________________________________________________________
dense_9 (Dense)              (None, 5000, 25)          1275      
_________________________________________________________________
dropout_4 (Dropout)          (None, 5000, 25)         

In [13]:
ann_model.compile(optimizer = 'adam', 
                  loss = tf.keras.losses.BinaryCrossentropy(), metrics=[tf.keras.metrics.BinaryAccuracy(name='accuracy', dtype=None, threshold=0.5),tf.keras.metrics.Recall(name='Recall'),tf.keras.metrics.Precision(name='Precision'), 
                    tf.keras.metrics.AUC(
        num_thresholds=200,
        curve="ROC",
        summation_method="interpolation",
        name="AUC",
        dtype=None,
        thresholds=None,
        multi_label=True,
        label_weights=None,)])

In [14]:
def train_model(train_dataset,val_dataset,model,epochs,batch_size):
    s1 = datetime.datetime.now()
    train_prefetch = train_dataset.batch(batch_size).prefetch(tf.data.experimental.AUTOTUNE)
    train_cache = train_prefetch.cache()
    val_dataset = val_dataset.batch(batch_size).cache()
    model.fit(train_cache,epochs=epochs,validation_data = val_dataset,callbacks=[callback])#add validation dataset
    s2 = datetime.datetime.now()
    print(s2-s1)
    return model

In [15]:
model_annabell = train_model(train_dataset,val_dataset,ann_model,50,128)

Epoch 1/50
     32/Unknown - 32s 929ms/step - loss: 0.4985 - accuracy: 0.8000 - Recall: 0.0000e+00 - Precision: 0.0000e+00 - AUC: 0.5529

KeyboardInterrupt: 

In [16]:
cnn_model = tf.keras.models.Sequential()
cnn_model.add(tf.keras.Input(shape=(5000,12)))
cnn_model.add(tf.keras.layers.Conv1D(filters = 4, kernel_size = 21, padding='same',activation = 'relu'))
cnn_model.add(tf.keras.layers.AvgPool1D(pool_size=1,strides=4,padding='same'))

cnn_model.add(tf.keras.layers.Conv1D(filters = 4, kernel_size = 23, padding='same',activation = 'relu'))
cnn_model.add(tf.keras.layers.AvgPool1D(pool_size=1,strides=4,padding='same'))

cnn_model.add(tf.keras.layers.Conv1D(filters = 4, kernel_size = 25, padding='same',activation = 'relu'))
cnn_model.add(tf.keras.layers.AvgPool1D(pool_size=1,strides=4,padding='same'))

cnn_model.add(tf.keras.layers.Conv1D(filters = 4, kernel_size = 27, padding='same',activation = 'relu'))

cnn_model.add(tf.keras.layers.Flatten())

cnn_model.add(tf.keras.layers.Dense(128,activation='relu'))
cnn_model.add(tf.keras.layers.Dropout(0.3))
cnn_model.add(tf.keras.layers.Dense(64,activation='relu'))
cnn_model.add(tf.keras.layers.Dropout(0.3))
cnn_model.add(tf.keras.layers.Dense(32,activation='relu'))
cnn_model.add(tf.keras.layers.Dropout(0.3))

cnn_model.add(tf.keras.layers.Dense(5,activation='softmax'))

In [17]:
cnn_model.compile(optimizer = 'adam', 
                  loss = tf.keras.losses.BinaryCrossentropy(), metrics=[tf.keras.metrics.BinaryAccuracy(name='accuracy', dtype=None, threshold=0.5),tf.keras.metrics.Recall(name='Recall'),tf.keras.metrics.Precision(name='Precision'), 
                    tf.keras.metrics.AUC(
        num_thresholds=200,
        curve="ROC",
        summation_method="interpolation",
        name="AUC",
        dtype=None,
        thresholds=None,
        multi_label=True,
        label_weights=None,)])

In [18]:
model_cyberpunk = train_model(train_dataset,val_dataset,cnn_model,50,128)

Epoch 1/50
249/249 [==============================] - 53s 207ms/step - loss: 0.4799 - accuracy: 0.8035 - Recall: 0.0322 - Precision: 0.5751 - AUC: 0.6055 - val_loss: 0.4158 - val_accuracy: 0.8134 - val_Recall: 0.1007 - val_Precision: 0.7500 - val_AUC: 0.7838
Epoch 2/50
249/249 [==============================] - 49s 197ms/step - loss: 0.4198 - accuracy: 0.8127 - Recall: 0.1116 - Precision: 0.6977 - AUC: 0.7626 - val_loss: 0.3954 - val_accuracy: 0.8116 - val_Recall: 0.1310 - val_Precision: 0.6416 - val_AUC: 0.7824
Epoch 3/50
249/249 [==============================] - 48s 192ms/step - loss: 0.3641 - accuracy: 0.8378 - Recall: 0.2755 - Precision: 0.7606 - AUC: 0.8285 - val_loss: 0.3740 - val_accuracy: 0.8419 - val_Recall: 0.3301 - val_Precision: 0.7322 - val_AUC: 0.7823
Epoch 4/50
249/249 [==============================] - 52s 208ms/step - loss: 0.3152 - accuracy: 0.8592 - Recall: 0.3816 - Precision: 0.8164 - AUC: 0.8597 - val_loss: 0.3771 - val_accuracy: 0.8464 - val_Recall: 0.3515 - val_

In [22]:
test_dataset = test_dataset.batch(200)
model_cyberpunk.evaluate(test_dataset)

InvalidArgumentError:  Cannot batch tensors with different shapes in component 0. First element had shape [1000,5000,12] and element 4 had shape [276,5000,12].
	 [[node IteratorGetNext (defined at <ipython-input-14-d6ecb4a1ec79>:6) ]] [Op:__inference_test_function_13393]

Function call stack:
test_function
